# Project: Prompt Engineering with LangChain and ChatGPT for real-world tasks

In this notebook you will leverage ChatGPT and LangChain to solve and do a few mini-projects based on some real-world scenarios:

- Mini-Project 1: Review Analyst
- Mini-Project 2: Research Paper Analyst
- Mini-Project 3: Social Media Marketing Analyst
- Mini-Project 4: IT Support Analyst

___[Created By: Dipanjan (DJ)](https://www.linkedin.com/in/dipanjans/)___

## Install OpenAI and LangChain dependencies

In [0]:
!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.8/431.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 36.0 MB/s eta 0:00:00
     ━━━━━━

In [0]:
dbutils.library.restartPython()

## Enter API Tokens

#### Enter your Open AI Key here

You can get the key from [here](https://platform.openai.com/api-keys) after creating an account or signing in

In [0]:
from getpass import getpass

OPENAI_KEY = getpass('Please enter your Open AI API Key here: ')

Please enter your Open AI API Key here:  [REDACTED]

## Setup necessary system environment variables

In [0]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

## Load Necessary Dependencies and ChatGPT LLM

In [0]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [0]:
chatgpt = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.0)

## Mini-Project 1: Review Analyst

You are building an AI system to be able to look at customer reviews and do some complex analysis. for each review get ChatGPT to do the following:

  - Summarize the review. The summary should be at most 3 lines.
  - Highlight both the positives and negatives
  - Display the overall sentiment of the review (positive, negative, neutral)
  - Display a list of 3 - 5 emotions expressed by the customer in the review
  - If the sentiment is positive or neutral write an email and thank them for the review
  - If the sentiment is negative apologize and write an email with an appropriate response

Try to get the response in a nice structured format using an output parser

### Access Customer Reviews

In [0]:
reviews = [
    f"""
    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    """,
    f"""
    Purchased a new gaming keyboard because of its rave reviews about responsiveness and backlighting.
    It hasn't disappointed. The keys have a satisfying click and the LED colors are vibrant,
    enhancing my gaming experience significantly. Price-wise, it's quite competitive,
    and I feel like I got a good deal. The delivery was swift, and it came well-protected,
    ensuring no damage during transport.
    """,
    f"""
    Ordered a set of wireless earbuds for running, and they've been a letdown.
    The sound constantly cuts out, and the fit is uncomfortable after only a few minutes of use.
    They advertised a 12-hour battery life, but I'm barely getting four hours.
    Considering the cost, I expected better quality and performance.
    They did arrive on time, but the positives end there. I'm already looking into a return.
    """,
    f"""
    The tablet stand I bought was touted as being sturdy and adjustable,
    but it's anything but. It wobbles with the slightest touch,
    and the angles are not holding up as promised. It feels like a breeze could knock it over.
    It was also pricier than others I've seen, which adds to the disappointment.
    It did arrive promptly, but what's the use if the product doesn't meet basic expectations?
    """,
    f"""
    Needed a new kitchen blender, but this model has been a nightmare.
    It's supposed to handle various foods, but it struggles with anything tougher than cooked vegetables.
    It's also incredibly noisy, and the 'easy-clean' feature is a joke; food gets stuck under the blades constantly.
    I thought the brand meant quality, but this product has proven me wrong.
    Plus, it arrived three days late. Definitely not worth the expense.
    """
]

### Define Output Parser

In [0]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# Define your desired data structure - like a python data class.
class ReviewAnalysisResponse(BaseModel):
    summary: str = Field(description="A brief summary of the customer review with maximum 3 lines")
    positives: list = Field(description="A list showing the positives mentioned by the customer in the review if any - max 3 points")
    negatives: list = Field(description="A list showing the negatives mentioned by the customer in the review if any - max 3 points")
    sentiment: str = Field(description="One word showing the sentiment of the review - positive, negative or neutral")
    emotions: list = Field(description="A list of 3 - 5 emotions expressed by the customer in the review")
    email: str = Field(description="Detailed email to the customer based on the sentiment")

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=ReviewAnalysisResponse)

### Create the input prompt for the LLM

In [0]:
# create the final prompt with formatting instructions from the parser
prompt_txt = """
             Analyze the given customer review below and generate the response based on the instructions
             mentioned below in the format instructions.
             Also remember to write a detailed email response for the email field based on these conditions:
               - email should be addressed to Dear Customer and signed with Service Agent
               - thank them if the review is positive or neutral
               - apologize if the review is negative

             Format Instructions:
             {format_instructions}

             Review:
             {review}
            """
prompt = PromptTemplate(
    template=prompt_txt,
    input_variables=["review"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

### Create a LCEL LLM Chain

In [0]:
# create a simple LCEL chain to take the prompt, pass it to the LLM, enforce response format using the parser
chain = (prompt
           |
         chatgpt
           |
         parser)

### Format the input reviews

In [0]:
reviews_formatted = [{'review': review} for review in reviews]
reviews_formatted[0]

{'review': "\n    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.\n    The sound quality is impressively clear with just the right amount of bass.\n    It's also waterproof, which tested true during a recent splashing incident.\n    Though it's compact, the volume can really fill the space.\n    The price was a bargain for such high-quality sound.\n    Shipping was also on point, arriving two days early in secure packaging.\n    "}

### Get responses from the LLM

In [0]:
responses = chain.map().invoke(reviews_formatted)

### View LLM responses

In [0]:
responses[0]

ReviewAnalysisResponse(summary='The customer is very pleased with the Bluetooth speaker, highlighting its sound quality, waterproof feature, and compact design. They also appreciated the early shipping and secure packaging.', positives=['Impressive sound quality with clear audio and good bass', 'Waterproof feature tested successfully', 'Compact design with powerful volume'], negatives=[], sentiment='positive', emotions=['satisfaction', 'happiness', 'excitement'], email="Dear Customer,\n\nThank you for your wonderful review! We're thrilled to hear that you are enjoying your new Bluetooth speaker and that it met your expectations in terms of sound quality, waterproof capability, and compact design. It's great to know that the early shipping and secure packaging added to your positive experience.\n\nIf you have any further questions or need assistance, feel free to reach out. We appreciate your support and hope you have many fantastic beach outings with your speaker!\n\nBest regards,\nSer

In [0]:
responses[0].dict()

/root/.ipykernel/2256/command-3389206174236837-632561186:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  responses[0].dict()


{'summary': 'The customer is very pleased with the Bluetooth speaker, highlighting its sound quality, waterproof feature, and compact design. They also appreciated the early shipping and secure packaging.',
 'positives': ['Impressive sound quality with clear audio and good bass',
  'Waterproof feature tested successfully',
  'Compact design with powerful volume'],
 'negatives': [],
 'sentiment': 'positive',
 'emotions': ['satisfaction', 'happiness', 'excitement'],
 'email': "Dear Customer,\n\nThank you for your wonderful review! We're thrilled to hear that you are enjoying your new Bluetooth speaker and that it met your expectations in terms of sound quality, waterproof capability, and compact design. It's great to know that the early shipping and secure packaging added to your positive experience.\n\nIf you have any further questions or need assistance, feel free to reach out. We appreciate your support and hope you have many fantastic beach outings with your speaker!\n\nBest regards,

In [0]:
for response in responses:
  for k,v in response.dict().items():
    print(f'{k}:\n{v}')
  print('-----')
  print('\n')

summary:
The customer is very pleased with the Bluetooth speaker, highlighting its sound quality, waterproof feature, and compact design. They also appreciated the early shipping and secure packaging.
positives:
['Impressive sound quality with clear audio and good bass', 'Waterproof feature tested successfully', 'Compact design with powerful volume']
negatives:
[]
sentiment:
positive
emotions:
['satisfaction', 'happiness', 'excitement']
email:
Dear Customer,

Thank you for your wonderful review! We're thrilled to hear that you are enjoying your new Bluetooth speaker and that it met your expectations in terms of sound quality, waterproof capability, and compact design. It's great to know that the early shipping and secure packaging added to your positive experience.

If you have any further questions or need assistance, feel free to reach out. We appreciate your support and hope you have many fantastic beach outings with your speaker!

Best regards,
Service Agent
-----


summary:
The cu

/root/.ipykernel/2256/command-3389206174236838-2992996444:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  for k,v in response.dict().items():


In [0]:
import pandas as pd

pd.DataFrame(response.dict() for response in responses)

/root/.ipykernel/2256/command-3389206174236839-1175396765:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  pd.DataFrame(response.dict() for response in responses)


,summary,positives,negatives,sentiment,emotions,email
0,The customer is very pleased with the Bluetoot...,[Impressive sound quality with clear audio and...,[],positive,"[satisfaction, happiness, excitement]","Dear Customer,\n\nThank you for your wonderful..."
1,The customer is highly satisfied with their ne...,"[Responsive keys with a satisfying click, Vibr...",[],positive,"[satisfaction, excitement, contentment]","Dear Customer,\n\nThank you for your wonderful..."
2,The customer is disappointed with the wireless...,[Arrived on time],"[Sound constantly cuts out, Uncomfortable fit ...",negative,"[disappointment, frustration, dissatisfaction]","Dear Customer,\n\nThank you for your feedback ..."
3,The customer is disappointed with the tablet s...,[Arrived promptly],"[Wobbles with the slightest touch, Angles do n...",negative,"[disappointment, frustration, dissatisfaction]","Dear Customer,\n\nThank you for your feedback ..."
4,The customer had a negative experience with th...,[],"[Struggles with tougher foods, Incredibly nois...",negative,"[frustration, disappointment, anger]","Dear Customer,\n\nThank you for taking the tim..."


## Mini-Project 2: Research Paper Analyst

Make ChatGPT act as an AI expert and transform the given research paper abstract based on the nature of the audience mentioned below.

- Short summary of maximum 10 lines for a general audience
- Detailed report for a healthcare company. Have bullet points for pros and cons of ethics in Generative AI as mentioned in the paper
- Detailed report for a generative AI company solving healthcare problems. Have bullet points for key points mentioned for Generative AI for text, images and structured data based healthcare

Try to use `ChatPromptTemplate` so you can have a conversation with ChatGPT for each of the above tasks using conversational prompting

### Access the Research Paper Abstract

In [0]:
paper_abstract = f"""
The widespread use of ChatGPT and other emerging technology powered by generative
artificial intelligence (AI) has drawn much attention to potential ethical issues, especially in
high-stakes applications such as healthcare.1–3 However, less clear is how to resolve such
issues beyond following guidelines and regulations that are still under discussion and
development. On the other hand, other types of generative AI have been used to synthesize
images and other types of data for research and practical purposes, which have resolved some
ethical issues and exposed other ethical issues,4,5 but such technology is less often the focus
of ongoing ethical discussions. Here we highlight gaps in current ethical discussions of
generative AI via a systematic scoping review of relevant existing research in healthcare, and
reduce the gaps by proposing an ethics checklist for comprehensive assessment and
transparent documentation of ethical discussions in generative AI development. While the
checklist can be readily integrated into the current peer review and publication system to
enhance generative AI research, it may also be used in broader settings to disclose ethicsrelated considerations in generative AI-powered products (or real-life applications of such
products) to help users establish reasonable trust in their capabilities.

Current ethical discussions on generative AI in healthcare
We conducted a systematic scoping review to analyse current ethical discussions on
generative AI in healthcare. Our search in four major academic research databases for
relevant publications from January 2013 to July 2023 yielded 2859 articles (see Methods for
detailed search strategy and Supplementary Figure S1 for the PRISMA flow diagram), of
which 193 articles were included for analysis based on application data modality (text, image,
or structured data), ethical issues discussed, generative AI involved, and whether generative
AI causes or offers technical solutions for issues raised.

Generative AI for text data-based healthcare
Forty-one of the 193 articles discussed ethical considerations pertaining to generative AI
applications for text data, with 20 articles describing methodological developments or
applications of generative AI and the other 21 articles describing review-type works on this
topic. Although some of these review-type articles used the general term “generative AI”, the
main body and supporting evidence focused on LLMs. Twenty-nine articles had in-depth
discussions on ethical issues, whereas the other 12 articles only briefly touched on some
ethical aspects.
Among the 41 articles, 29 articles focused on discussing ethical issues caused by LLMs (and
specifically by GPT in 16 of the articles), covering a wide range of application scenarios and
considered the application of all 10 ethical principles identified in the review (see Figure 1),
as well as other less discussed concerns such as human-AI interaction, and the rights of
LLMs to be considered as co-authors in scientific papers. One paper only commented briefly
on the need for ethical considerations in LLMs and is summarised in the “Others” category.
Although all ethical principles are equally important, some are discussed more often than
others, e.g., non-maleficence (also referred to in the literature as ‘benevolence’), equity, and
privacy.
Fifteen of the 41 articles aimed to resolve some existing ethical issues (for example,
confidentiality of medical data) by using LLMs and other generative AI (e.g., GAN,
autoencoder or diffusion), such as, to reduce privacy concerns by generating synthetic
medical text, to reduce disparity by providing accessible services and assistance, to detect
health-related misinformation, to generate trusted content, and to improve accountability or
transparency over existing approaches. While most articles focused on either identifying
ethical issues caused by generative AI or proposing generative AI-based solutions, three
articles discussed both to provide a more balanced perspective.

Generative AI for image and structured data-based healthcare
Unlike the diverse application scenarios of generative AI based on text data, for image and
structured data, this use of generative AI focuses on data synthesis and encryption. Hence the
majority of articles discussed the methodological developments of generative AI as giving
rise to a more distinctive and focused set of ethical issues.
5
Notably, of the 98 articles on image data and 58 articles on structured data, more than half
(n=63 for image data and n=33 for structured data) only mentioned ethical considerations as a
brief motivation for methodological developments or as a general discussion point. The rest
included more in-depth discussions or evaluations of ethical issues. Among these 155 articles
(as one article covered multiple modalities), 11 articles were review-type work, where 10
articles reviewed methods that mentioned one or two ethical perspectives, and only one
article24 discussed detailed ethical concerns on generative AI applications.
Resolving privacy issues was the main aim of articles for these two data modalities (n=74 for
image data and n=50 for structured data; see Figure 1), predominantly by generating synthetic
data using GAN. Eight articles on image data and 9 articles on structured data used
generative AI to reduce bias, e.g., by synthesizing data for under-represented subgroups in
existing databases. For both data modalities, we did not see explicit discussions on resolving
autonomy, integrity, or morality issues using generative AI, and for structured data the articles
additionally lacked discussions on trust or transparency.
Only 11 articles for image data selectively discussed some ethical issues that generative AI
can give rise to, without specific discussions regarding autonomy, integrity, or morality. For
structured data, only 4 articles discussed equity, privacy, or data security issues caused by
generative AI. Only two articles on structured data included both the cause and resolving
perspectives by discussing ethical issues that may arise from limitations of methods
proposed, specifically bias induced when synthesizing data in order to resolve privacy issues.
"""

### Create a prompt template for paper analysis and transformation

In [0]:
SYS_PROMPT = """
Act as a Artificial Intelligence Expert.
Transform the input research paper abstract given below
based on the instruction input by the user.
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        ("human", "{instruction}"),
    ]
)

In [0]:
prompt

ChatPromptTemplate(input_variables=['instruction'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nAct as a Artificial Intelligence Expert.\nTransform the input research paper abstract given below\nbased on the instruction input by the user.\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['instruction'], input_types={}, partial_variables={}, template='{instruction}'), additional_kwargs={})])

### Create a simple LCEL LLM Chain

In [0]:
chain = (prompt
            |
         chatgpt)

### Generate the first summary report

In [0]:
from langchain_core.messages import HumanMessage

prompt_txt = f"""
Based on the following research paper abstract,
create the summary report of maximum 10 lines
for a general audience

Abstract:
{paper_abstract}
"""
messages = [HumanMessage(content=prompt_txt)]
user_instruction = {'instruction': messages}

response = chain.invoke(user_instruction)
messages.append(response)

In [0]:
user_instruction

{'instruction': [HumanMessage(content='\nBased on the following research paper abstract,\ncreate the summary report of maximum 10 lines\nfor a general audience\n\nAbstract:\n\nThe widespread use of ChatGPT and other emerging technology powered by generative\nartificial intelligence (AI) has drawn much attention to potential ethical issues, especially in\nhigh-stakes applications such as healthcare.1–3 However, less clear is how to resolve such\nissues beyond following guidelines and regulations that are still under discussion and\ndevelopment. On the other hand, other types of generative AI have been used to synthesize\nimages and other types of data for research and practical purposes, which have resolved some\nethical issues and exposed other ethical issues,4,5 but such technology is less often the focus\nof ongoing ethical discussions. Here we highlight gaps in current ethical discussions of\ngenerative AI via a systematic scoping review of relevant existing research in healthcare, 

In [0]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

# Initialize the chat model
chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Define system prompt for context
sys_prompt = """Act as an Artificial Intelligence Expert. 
Transform the input research paper abstract based on the instruction input by the user."""

# Create a prompt template
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", sys_prompt),
    ("human", "{instruction}")
])

# Define reusable function for invoking chat with instruction
def generate_response(instruction_text: str) -> str:
    messages = [HumanMessage(content=instruction_text)]
    user_instruction = {'instruction': messages}
    return chatgpt.invoke(chat_prompt.format_messages(**user_instruction))

# Example research paper abstract
paper_abstract = paper_abstract

# First task: Create a summary report
summary_instruction = f"""
Based on the following research paper abstract, create a summary report of maximum 10 lines for a general audience.

Abstract:
{paper_abstract}
"""
response = generate_response(summary_instruction)
print(response.content)

The research paper examines the ethical implications of generative artificial intelligence (AI), particularly in healthcare settings. It highlights the growing use of technologies like ChatGPT and the need for clearer ethical guidelines as these tools become more prevalent. A systematic review of existing literature identified gaps in current ethical discussions, leading to the proposal of an ethics checklist. This checklist aims to enhance transparency and accountability in generative AI research and applications. The study found that while many articles addressed ethical issues related to text-based generative AI, discussions on image and structured data were less comprehensive. Overall, the paper emphasizes the importance of integrating ethical considerations into the development and use of generative AI technologies to foster trust and responsible usage.


In [0]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

# Initialize the chat model
chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Define system-level prompt for context
SYS_PROMPT = """Act as an Artificial Intelligence Expert. 
Transform the input research paper abstract based on the instruction input by the user."""

# Create a chat prompt template
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", SYS_PROMPT),
    ("human", "{instruction}")
])

# Build a chain for processing messages
chain = chat_prompt | chatgpt

# Define reusable function to interact with the chain
def generate_response(instruction_text: str) -> str:
    # Create a human message
    messages = [HumanMessage(content=instruction_text)]
    # Wrap messages in instruction dictionary as required
    user_instruction = {'instruction': messages}
    # Invoke the chain and return the response
    response = chain.invoke(user_instruction)
    return response.content

# Example research paper abstract
paper_abstract = """(Long abstract content here)"""

# First task: Create a summary report
summary_instruction = f"""
Based on the following research paper abstract, create a summary report of maximum 10 lines for a general audience.

Abstract:
{paper_abstract}
"""
summary_response = generate_response(summary_instruction)
print("Summary Report:\n", summary_response)

# Second task: Detailed report for a healthcare company
detailed_instruction = f"""
Using only the research paper abstract provided earlier, create a detailed report for a healthcare company.
Include bullet points (3 max) for pros and cons of ethics in Generative AI.
"""
detailed_response = generate_response(detailed_instruction)
print("Detailed Report:\n", detailed_response)

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Define system-level behavior for AI
sys_prompt = """Act as an Artificial Intelligence Expert. Transform the input research paper abstract based on the instruction input by the user."""
messages = [SystemMessage(content=sys_prompt)]

# First user instruction
paper_abstract = paper_abstract
prompt_txt = f"""
Based on the following research paper abstract, create the summary report of maximum 10 lines for a general audience

Abstract:
{paper_abstract}
"""
messages.append(HumanMessage(content=prompt_txt))
response = chatgpt.invoke(messages)
print(response)

content='The increasing use of generative artificial intelligence (AI), such as ChatGPT, raises important ethical concerns, particularly in healthcare. This research highlights gaps in current ethical discussions and proposes an ethics checklist to guide the development and application of generative AI technologies. A systematic review of 2,859 articles from 2013 to 2023 revealed that while many studies address ethical issues related to text-based generative AI, discussions around image and structured data applications are less comprehensive. Key ethical principles, including privacy and equity, are often emphasized, but many articles only briefly mention ethical considerations. The proposed checklist aims to enhance transparency and trust in generative AI applications, ensuring ethical considerations are integrated into research and real-world use.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 144, 'prompt_tokens': 1275, 'total_tokens': 1

In [0]:
print(response.content)

The research paper examines the ethical implications of generative artificial intelligence (AI), particularly in healthcare settings. It highlights the growing use of technologies like ChatGPT and the need for clearer ethical guidelines as these tools become more prevalent. A systematic review of existing literature identified gaps in current ethical discussions, leading to the proposal of an ethics checklist. This checklist aims to enhance transparency and accountability in generative AI research and applications. The study found that while many articles address ethical issues related to text-based generative AI, discussions on image and structured data are less comprehensive. Overall, the paper emphasizes the importance of integrating ethical considerations into the development and use of generative AI technologies.


In [0]:
response.content

'The research paper examines the ethical implications of generative artificial intelligence (AI), particularly in healthcare settings. It highlights the growing use of technologies like ChatGPT and the need for clearer ethical guidelines as these tools become more prevalent. A systematic review of existing literature identified gaps in current ethical discussions, leading to the proposal of an ethics checklist. This checklist aims to enhance transparency and accountability in generative AI research and applications. The study found that while many articles address ethical issues related to text-based generative AI, discussions on image and structured data are less comprehensive. Overall, the paper emphasizes the importance of integrating ethical considerations into the development and use of generative AI technologies.'

In [0]:
messages

[HumanMessage(content='\nBased on the following research paper abstract,\ncreate the summary report of maximum 10 lines\nfor a general audience\n\nAbstract:\n\nThe widespread use of ChatGPT and other emerging technology powered by generative\nartificial intelligence (AI) has drawn much attention to potential ethical issues, especially in\nhigh-stakes applications such as healthcare.1–3 However, less clear is how to resolve such\nissues beyond following guidelines and regulations that are still under discussion and\ndevelopment. On the other hand, other types of generative AI have been used to synthesize\nimages and other types of data for research and practical purposes, which have resolved some\nethical issues and exposed other ethical issues,4,5 but such technology is less often the focus\nof ongoing ethical discussions. Here we highlight gaps in current ethical discussions of\ngenerative AI via a systematic scoping review of relevant existing research in healthcare, and\nreduce the 

### Generate the second summary report

Here we add the previous LLM response and the new instructions to the list of messages and send the whole thing to the LLM so it has access to the historical conversation

In [0]:
prompt_txt = f"""
Use only the research paper abstract from earlier and create a detailed report for a healthcare company.
In the report, also include bullet points (3 max) for pros and cons of ethics in Generative AI
"""
messages.append(HumanMessage(content=prompt_txt))
user_instruction = {'instruction': messages}
response = chain.invoke(user_instruction)
messages.append(response)

In [0]:
print(response.content)

**Detailed Report on Ethical Considerations in Generative AI for Healthcare**

**Introduction:**
The rapid adoption of generative artificial intelligence (AI) technologies, such as ChatGPT, has raised significant ethical concerns, particularly in high-stakes fields like healthcare. This report synthesizes findings from a systematic review of existing literature on the ethical implications of generative AI in healthcare, highlighting current gaps and proposing a comprehensive ethics checklist to guide future developments.

**Key Findings:**
- **Ethical Gaps Identified:** The review revealed that while there is a substantial amount of literature discussing ethical issues related to text-based generative AI, similar discussions for image and structured data applications are less comprehensive. This inconsistency highlights a need for more robust ethical frameworks across all modalities of generative AI.
  
- **Proposed Ethics Checklist:** To address these gaps, the report proposes an ethi

In [0]:
messages

[HumanMessage(content='\nBased on the following research paper abstract,\ncreate the summary report of maximum 10 lines\nfor a general audience\n\nAbstract:\n\nThe widespread use of ChatGPT and other emerging technology powered by generative\nartificial intelligence (AI) has drawn much attention to potential ethical issues, especially in\nhigh-stakes applications such as healthcare.1–3 However, less clear is how to resolve such\nissues beyond following guidelines and regulations that are still under discussion and\ndevelopment. On the other hand, other types of generative AI have been used to synthesize\nimages and other types of data for research and practical purposes, which have resolved some\nethical issues and exposed other ethical issues,4,5 but such technology is less often the focus\nof ongoing ethical discussions. Here we highlight gaps in current ethical discussions of\ngenerative AI via a systematic scoping review of relevant existing research in healthcare, and\nreduce the 

### Generate the third summary report

Here we add the previous LLM response and the new instructions to the list of messages and send the whole thing to the LLM so it has access to the historical conversation

In [0]:
prompt_txt = f"""
Use only the research paper abstract from earlier and create a detailed report for a generative AI company solving healthcare problems.
In the report also include sections for key points mentioned around Generative AI for text, images and structured data based healthcare
"""
messages.append(HumanMessage(content=prompt_txt))
user_instruction = {'instruction': messages}
response = chain.invoke(user_instruction)

In [0]:
print(response.content)

**Detailed Report on Generative AI in Healthcare: Ethical Considerations and Applications**

**Introduction:**
The increasing integration of generative artificial intelligence (AI) technologies, such as ChatGPT, into healthcare has sparked significant ethical discussions. This report synthesizes findings from a systematic review of existing literature on the ethical implications of generative AI in healthcare, focusing on applications involving text, images, and structured data.

**Key Findings:**
- **Ethical Gaps Identified:** The review highlights a disparity in the depth of ethical discussions across different modalities of generative AI. While text-based applications have been extensively analyzed, discussions surrounding image and structured data applications are less comprehensive, indicating a need for a more uniform ethical framework.

- **Proposed Ethics Checklist:** To bridge these gaps, an ethics checklist has been proposed. This tool aims to facilitate the assessment and do

## Mini-Project 3: Social Media Marketing Analyst

You have the technical fact sheets of one smartphone. Try some iterative prompt engineering and do the following:

1. Generate marketing product description for the smartphone

2. Custom product description which has the following:

```
The description should follow this format:

Product Name: <Name of the smartphone>
​
Description: <Brief Overview of the features>
​
Product Specifications:
<Table with key product feature specifications>
​
The description should focus on the most important features
a customer might look for in a phone including the foldable display screen, processing power, RAM, camera and battery life.
​
After the description, the table should have the
key specifications of the product. It should have two columns.
The first column should have 'Feature'
and the second column should have 'Specification'
and try to put exact numeric values for features if they exist.
Only put these features in the table - foldable display screen, processing power, RAM, camera and battery life
```

3. Custom product description focusing on specific aspects like display, camera and in less than 60 words

### Access the product factsheet data

In [0]:
fact_sheet_mobile = """
PRODUCT NAME
Samsung Galaxy Z Fold4 5G Black
​
PRODUCT OVERVIEW
Stands out. Stands up. Unfolds.
The Galaxy Z Fold4 does a lot in one hand with its 15.73 cm(6.2-inch) Cover Screen.
Unfolded, the 19.21 cm(7.6-inch) Main Screen lets you really get into the zone.
Pushed-back bezels and the Under Display Camera means there's more screen
and no black dot getting between you and the breathtaking Infinity Flex Display.
Do more than more with Multi View. Whether toggling between texts or catching up
on emails, take full advantage of the expansive Main Screen with Multi View.
PC-like power thanks to Qualcomm Snapdragon 8+ Gen 1 processor in your pocket,
transforms apps optimized with One UI to give you menus and more in a glance
New Taskbar for PC-like multitasking. Wipe out tasks in fewer taps. Add
apps to the Taskbar for quick navigation and bouncing between windows when
you're in the groove.4 And with App Pair, one tap launches up to three apps,
all sharing one super-productive screen
Our toughest Samsung Galaxy foldables ever. From the inside out,
Galaxy Z Fold4 is made with materials that are not only stunning,
but stand up to life's bumps and fumbles. The front and rear panels,
made with exclusive Corning Gorilla Glass Victus+, are ready to resist
sneaky scrapes and scratches. With our toughest aluminum frame made with
Armor Aluminum, this is one durable smartphone.
World’s first water resistant foldable smartphones. Be adventurous, rain
or shine. You don't have to sweat the forecast when you've got one of the
world's first water-resistant foldable smartphones.
​
PRODUCT SPECS
OS - Android 12.0
RAM - 12 GB
Product Dimensions - 15.5 x 13 x 0.6 cm; 263 Grams
Batteries - 2 Lithium Ion batteries required. (included)
Item model number - SM-F936BZKDINU_5
Wireless communication technologies - Cellular
Connectivity technologies - Bluetooth, Wi-Fi, USB, NFC
GPS - True
Special features - Fast Charging Support, Dual SIM, Wireless Charging, Built-In GPS, Water Resistant
Other display features - Wireless
Device interface - primary - Touchscreen
Resolution - 2176x1812
Other camera features - Rear, Front
Form factor - Foldable Screen
Colour - Phantom Black
Battery Power Rating - 4400
Whats in the box - SIM Tray Ejector, USB Cable
Manufacturer - Samsung India pvt Ltd
Country of Origin - China
Item Weight - 263 g
"""

### Create prompt template for the first advert

In [0]:
prompt_txt = """
Act as a marketing manager.
Your task is to help a marketing team create a
description for a retail website advert of a product based
on a technical fact sheet specifications for a mobile smartphone
​
Write a brief product description

Technical specifications:
{fact_sheet_mobile}
"""
chat_template = ChatPromptTemplate.from_template(prompt_txt)

### Use an LCEL LLM Chain to generate the first advert

In [0]:
chain = (chat_template
            |
         chatgpt)
response = chain.invoke({"fact_sheet_mobile": fact_sheet_mobile})

In [0]:
print(response.content)

**Product Description: Samsung Galaxy Z Fold4 5G - Phantom Black**

Unleash the power of innovation with the Samsung Galaxy Z Fold4 5G in stunning Phantom Black. This groundbreaking foldable smartphone redefines versatility, seamlessly transitioning from a compact 6.2-inch Cover Screen to an expansive 7.6-inch Main Screen that immerses you in your favorite content. With its breathtaking Infinity Flex Display, enjoy a truly edge-to-edge experience, free from distractions.

Powered by the Qualcomm Snapdragon 8+ Gen 1 processor and equipped with 12GB of RAM, the Galaxy Z Fold4 delivers PC-like performance right in your pocket. Effortlessly multitask with the new Taskbar and Multi View features, allowing you to toggle between apps and complete tasks with ease. Whether you're catching up on emails or enjoying your favorite shows, this smartphone is designed to keep you in the zone.

Durability meets elegance with the Galaxy Z Fold4, featuring Corning Gorilla Glass Victus+ and Armor Aluminum

In [0]:
from IPython.display import display, Markdown
display(Markdown(response.content))

**Product Description: Samsung Galaxy Z Fold4 5G - Phantom Black**

Unleash the power of innovation with the Samsung Galaxy Z Fold4 5G in stunning Phantom Black. This groundbreaking foldable smartphone redefines versatility, seamlessly transitioning from a compact 6.2-inch Cover Screen to an expansive 7.6-inch Main Screen that immerses you in your favorite content. With its breathtaking Infinity Flex Display, enjoy a truly edge-to-edge experience, free from distractions.

Powered by the Qualcomm Snapdragon 8+ Gen 1 processor and equipped with 12GB of RAM, the Galaxy Z Fold4 delivers PC-like performance right in your pocket. Effortlessly multitask with the new Taskbar and Multi View features, allowing you to toggle between apps and complete tasks with ease. Whether you're catching up on emails or enjoying your favorite shows, this smartphone is designed to keep you in the zone.

Durability meets elegance with the Galaxy Z Fold4, featuring Corning Gorilla Glass Victus+ and Armor Aluminum for enhanced protection against life's everyday bumps. Plus, it's one of the world's first water-resistant foldable smartphones, so you can embrace adventure without worrying about the weather.

With a powerful 4400mAh battery, fast charging support, and wireless charging capabilities, the Galaxy Z Fold4 is ready to keep up with your busy lifestyle. Experience the future of mobile technology today with the Samsung Galaxy Z Fold4 5G – where style meets functionality. 

**What's in the box:** SIM Tray Ejector, USB Cable. 

**Get yours now and redefine what a smartphone can do!**

### Create prompt template for the second advert

In [0]:
prompt_txt = """
Act as a marketing manager.
Your task is to help a marketing team create a
description for a retail website advert of a product based
on a technical fact sheet specifications for a mobile smartphone
​
The description should follow this format:

Product Name: <Name of the smartphone>
​
Description: <Brief Overview of the features>
​
Product Specifications:
<Table with key product feature specifications>
​
The description should focus on the most important features
a customer might look for in a phone including the foldable display screen, processing power, RAM, camera and battery life.
​
After the description, the table should have the
key specifications of the product. It should have two columns.
The first column should have 'Feature'
and the second column should have 'Specification'
and try to put exact numeric values for features if they exist.
Only put these features in the table - foldable display screen, processing power, RAM, camera and battery life

Technical specifications:
{fact_sheet_mobile}
"""
chat_template = ChatPromptTemplate.from_template(prompt_txt)

### Use an LCEL LLM Chain to generate the second advert

In [0]:
chain = (chat_template
            |
         chatgpt)
response = chain.invoke({"fact_sheet_mobile": fact_sheet_mobile})

In [0]:
print(response.content)

**Product Name:** Samsung Galaxy Z Fold4 5G Black

**Description:**  
Experience the future of mobile technology with the Samsung Galaxy Z Fold4 5G. This innovative smartphone features a stunning 7.6-inch Infinity Flex Display that unfolds to provide an immersive viewing experience, perfect for multitasking and entertainment. With its powerful Qualcomm Snapdragon 8+ Gen 1 processor and 12 GB of RAM, you can seamlessly run multiple apps and enjoy smooth performance. Capture every moment with the advanced camera system, and stay connected all day long with a robust 4400 mAh battery. Plus, with its water-resistant design and durable materials, the Galaxy Z Fold4 is built to withstand life's adventures.

**Product Specifications:**

| Feature                | Specification          |
|-----------------------|------------------------|
| Foldable Display Screen| 7.6 inches (Main), 6.2 inches (Cover) |
| Processing Power      | Qualcomm Snapdragon 8+ Gen 1 |
| RAM                   | 12 GB   

In [0]:
from IPython.display import display, Markdown
display(Markdown(response.content))

**Product Name:** Samsung Galaxy Z Fold4 5G Black

**Description:**  
Experience the future of mobile technology with the Samsung Galaxy Z Fold4 5G. This innovative smartphone features a stunning 7.6-inch Infinity Flex Display that unfolds to provide an immersive viewing experience, perfect for multitasking and entertainment. With its powerful Qualcomm Snapdragon 8+ Gen 1 processor and 12 GB of RAM, you can seamlessly run multiple apps and enjoy smooth performance. Capture every moment with the advanced camera system, and stay connected all day long with a robust 4400 mAh battery. Plus, with its water-resistant design and durable materials, the Galaxy Z Fold4 is built to withstand life's adventures.

**Product Specifications:**

| Feature                | Specification          |
|-----------------------|------------------------|
| Foldable Display Screen| 7.6 inches (Main), 6.2 inches (Cover) |
| Processing Power      | Qualcomm Snapdragon 8+ Gen 1 |
| RAM                   | 12 GB                  |
| Camera                | Rear: Triple Camera, Front: 10 MP |
| Battery Life          | 4400 mAh               |

### Create prompt template for the third advert

In [0]:
prompt_txt = """
Act as a marketing manager.
Your task is to help a marketing team create a
description for a retail website advert of a product based
on a technical fact sheet specifications for a mobile smartphone
​
Write a catchy product description with some emojis,
which uses at most 60 words
and focuses on the most important things about the smartphone
which might matter to users like display and camera

Technical specifications:
{fact_sheet_mobile}
"""
chat_template = ChatPromptTemplate.from_template(prompt_txt)

### Use an LCEL LLM Chain to generate the third advert

In [0]:
chain = (chat_template
            |
         chatgpt)
response = chain.invoke({"fact_sheet_mobile": fact_sheet_mobile})

In [0]:
print(response.content)

🌟 Unfold a new world with the Samsung Galaxy Z Fold4 5G! 📱✨ Experience stunning visuals on a 7.6-inch Infinity Flex Display and capture every moment with its advanced camera system. With lightning-fast performance and water resistance, this foldable powerhouse is built for adventure. Elevate your multitasking game and do more, effortlessly! 🚀💧


In [0]:
from IPython.display import display, Markdown
display(Markdown(response.content))

🌟 Unfold a new world with the Samsung Galaxy Z Fold4 5G! 📱✨ Experience stunning visuals on a 7.6-inch Infinity Flex Display and capture every moment with its advanced camera system. With lightning-fast performance and water resistance, this foldable powerhouse is built for adventure. Elevate your multitasking game and do more, effortlessly! 🚀💧

## Mini-Project 4 - IT Support Analyst

Ask ChatGPT to act as a IT support agent, process each customer IT ticket message and output the response in JSON with the following fields

```
orig_msg: The original customer message
orig_lang: Detected language of the customer message e.g. Spanish
category: 1-2 word describing the category of the problem
trans_msg: Translated customer message in English
response: Response to the customer in orig_lang
trans_response: Response to the customer in English
```

Try to use a JSON parser to get the responses in JSON for each ticket

### Define Output Parser

In [0]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field


# Define your desired data structure - like a python data class.
class ITSupportResponse(BaseModel):
    orig_msg: str = Field(description="The original customer IT support query message")
    orig_lang: str = Field(description="Detected language of the customer message e.g. Spanish")
    category: str = Field(description="1-2 word describing the category of the problem")
    trans_msg: str = Field(description="Translated customer IT support query message in English")
    response: str = Field(description="Response to the customer in their original language - orig_lang")
    trans_response: str = Field(description="Response to the customer in English")


parser = JsonOutputParser(pydantic_object=ITSupportResponse)

/databricks/python/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3508: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


### Create the input prompt for the LLM

In [0]:
# create the final prompt with formatting instructions from the parser
prompt_txt = """
             Act as an Information Technology (IT) customer support agent.
             For the IT support message mentioned below
             use the following output format when generating the output response

             Output format instructions:
             {format_instructions}

             Customer IT support message:
             {it_support_msg}
             """
prompt = PromptTemplate(
    template=prompt_txt,
    input_variables=["it_support_msg"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

### Create a LCEL LLM Chain

In [0]:
# create a simple LCEL chain to take the prompt, pass it to the LLM, enforce response format using the parser
llm_chain = (prompt
              |
            chatgpt
              |
            parser)

### Access Customer IT Support ticket data

In [0]:
it_support_queue = [
    "Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.",
    "Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.",
    "プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。",
    "Я не могу войти в систему учета времени, появляется сообщение об ошибке. Мне нужна помощь.",
    "Internet bağlantım çok yavaş ve bazen tamamen kesiliyor. Yardım eder misiniz?",
    "Не могу установить обновление безопасности. Появляется код ошибки. Помогите, пожалуйста."
]

formatted_msgs = [{"it_support_msg": msg}
                    for msg in it_support_queue]
formatted_msgs[0]

{'it_support_msg': 'Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.'}

### Get responses from the LLM

In [0]:
responses = llm_chain.map().invoke(formatted_msgs)

### View LLM responses

In [0]:
responses[0]

{'orig_msg': 'Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.',
 'orig_lang': 'Portuguese',
 'category': 'Sync Issue',
 'trans_msg': 'I cannot sync my contacts with the phone. I always receive a failure message.',
 'response': 'Por favor, verifique se você está conectado à internet e tente reiniciar o telefone. Se o problema persistir, considere desinstalar e reinstalar o aplicativo de contatos.',
 'trans_response': 'Please check if you are connected to the internet and try restarting the phone. If the problem persists, consider uninstalling and reinstalling the contacts app.'}

In [0]:
type(responses[0])

dict

In [0]:
import pandas as pd

df = pd.DataFrame(responses)
df

,orig_msg,orig_lang,category,trans_msg,response,trans_response
0,Não consigo sincronizar meus contatos com o te...,Portuguese,Sync Issue,I cannot sync my contacts with the phone. I al...,"Por favor, verifique se você está conectado à ...",Please check if you are connected to the inter...
1,Ho problemi a stampare i documenti da remoto. ...,Italian,Printing Issue,I have problems printing documents remotely. T...,Mi dispiace sapere che hai problemi a stampare...,I'm sorry to hear that you're having trouble p...
2,プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。,Japanese,Printer Issue,"I replaced the printer toner, but the print qu...",トナーを交換した後、印刷品質が低下することがあります。まず、プリンターのヘッドをクリーニング...,"After replacing the toner, print quality can s..."
3,"Я не могу войти в систему учета времени, появл...",Russian,Login Issue,"I cannot log into the time tracking system, an...","Пожалуйста, проверьте свои учетные данные и уб...",Please check your credentials and ensure you a...
4,Internet bağlantım çok yavaş ve bazen tamamen ...,Turkish,Internet Issue,My internet connection is very slow and someti...,Bağlantı sorunlarınızı çözmek için birkaç adım...,We can take a few steps to resolve your connec...
5,Не могу установить обновление безопасности. По...,Russian,Update Issue,I cannot install the security update. An error...,"Пожалуйста, проверьте, что у вас достаточно ме...",Please check that you have enough disk space t...


Try out more use-cases based on your own problems using what you learnt!